In [1]:
import warnings
import os
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras.layers import GlobalAveragePooling2D,GlobalMaxPooling2D
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

In [2]:
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import Input, Convolution2D, MaxPooling2D, Activation, concatenate, Dropout
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Input
from tensorflow.keras.utils import plot_model

In [3]:
trainpath = "../Trainingdata/splitpre2701515/train"
validpath = "../Trainingdata/splitpre2701515/val"
testpath = "../Trainingdata/splitpre2701515/test"
train_labels = os.listdir(trainpath)
num_classes=len(train_labels)

In [4]:
xsize=224
ysize=224

In [5]:
def Modelnet(input_shape=None, pooling=None, classes=None):

    img_input = Input(shape=input_shape)
    
    x = Convolution2D(64, (3, 3), strides=(2, 2), padding='valid', name='conv1')(img_input)
    x = BatchNormalization(name='BN1')(x)
    x = Activation('relu', name='relu_conv1')(x)

    x = Convolution2D(128, (3, 3), strides=(2, 2), padding='valid', name='conv2')(x)
    x = BatchNormalization(name='BN2')(x)
    x = Activation('relu', name='relu_conv2')(x)

    x = Convolution2D(128, (3, 3), strides=(2, 2), padding='valid', name='conv3')(x)
    x = BatchNormalization(name='BN3')(x)
    x = Activation('relu', name='relu_conv3')(x)

    x = Convolution2D(256, (3, 3), strides=(2, 2), padding='valid', name='conv4')(x)
    x = BatchNormalization(name='BN4')(x)
    x = Activation('relu', name='relu_conv4')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool3')(x)
    
    x = Dropout(0.5, name='drop1')(x)
    x = Convolution2D(num_classes, (1, 1), padding='valid', name='convf')(x)
    x = Activation('relu', name='relu_convf')(x)
    
    if pooling == 'avg':
        x = GlobalAveragePooling2D()(x)
    elif pooling=='max':
        x = GlobalMaxPooling2D()(x)
    elif pooling==None:
        pass
    else:
        raise ValueError("Unknown argument for 'pooling'=" + pooling)

    inputs = img_input
    preds = Activation('softmax', name='loss')(x)
    model = Model(inputs, preds, name='Modelnet')
    return model

In [6]:
model=Modelnet(input_shape=(xsize, ysize,3), pooling = 'avg', classes=num_classes)

In [7]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input)                               
train_generator=train_datagen.flow_from_directory(trainpath,
                                                  target_size=(xsize, ysize),
                                                  color_mode='rgb',
                                                  batch_size=16,
                                                  class_mode='categorical',
                                                  shuffle=True)

valid_datagen=ImageDataGenerator(preprocessing_function=preprocess_input)
validation_generator = valid_datagen.flow_from_directory(validpath,
                                                         target_size=(xsize, ysize),
                                                         color_mode='rgb',
                                                         batch_size=16,
                                                         class_mode='categorical',
                                                         shuffle=True)

test_datagen=ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_datagen.flow_from_directory(testpath,
                                                         target_size=(xsize, ysize),
                                                         color_mode='rgb',
                                                         batch_size=1,
                                                         class_mode=None,
                                                         shuffle=False) 

Found 796 images belonging to 5 classes.
Found 171 images belonging to 5 classes.
Found 173 images belonging to 5 classes.


In [8]:
learnrate=1e-3
# sgd = SGD(lr=learnrate,momentum=0.9,decay=0.00025) #SGD
opt=Adam(lr=learnrate, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])

In [9]:
step_size_train=train_generator.n//train_generator.batch_size
step_size_vali=validation_generator.n//validation_generator.batch_size
step_size_test=test_generator.n//test_generator.batch_size
print(step_size_train, step_size_vali, step_size_test)

49 10 173


In [10]:
history=model.fit_generator(generator=train_generator,
                            steps_per_epoch=step_size_train,
                            epochs=10,
                            validation_data=validation_generator,
                            validation_steps=step_size_vali)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
49/49 [==============================] - 29s 599ms/step - loss: 0.8743 - accuracy: 0.6513 - val_loss: 2.4413 - val_accuracy: 0.2000
Epoch 2/10
49/49 [==============================] - 43s 881ms/step - loss: 0.5610 - accuracy: 0.7731 - val_loss: 3.8132 - val_accuracy: 0.2250
Epoch 3/10
49/49 [==============================] - 29s 592ms/step - loss: 0.4074 - accuracy: 0.8577 - val_loss: 5.2641 - val_accuracy: 0.2062
Epoch 4/10
49/49 [==============================] - 30s 602ms/step - loss: 0.3401 - accuracy: 0.8731 - val_loss: 4.4014 - val_accuracy: 0.2188
Epoch 5/10
49/49 [==============================] - 30s 619ms/step - loss: 0.3076 - accuracy: 0.8949 - val_loss: 5.4865 - val_accuracy: 0.2062
Epoch 6/10
49/49 [==============================] - 30s 606ms/step - loss: 0.2259 - accuracy: 0.9333 - val_loss: 3.9908 - val_accuracy: 0.2750
Epoch 7/10
49/49 [==============================] - 29s 598ms/step

In [11]:
# prune the model----------------------------------------
orifile = 'ownmodel_ori.h5'
print('Saving model to: ', orifile)
models.save_model(model, orifile, include_optimizer=False)

Saving model to:  ownmodel_ori.h5


In [12]:
test_generator.reset()
start_time = time.time()
predo=model.predict_generator(test_generator,
                             steps=step_size_test,
                             verbose=1)
print("--- %s seconds ---" % (time.time() - start_time))

Instructions for updating:
Please use Model.predict, which supports generators.
173/173 [==============================] - 2s 11ms/step
--- 2.059000253677368 seconds ---


In [13]:
from tensorflow_model_optimization.sparsity import keras as sparsity

In [14]:
# Load the serialized model
loaded_model = models.load_model(orifile)
newepochs = 15
end_step = step_size_train * newepochs
print(end_step)

new_pruning_params = {
      'pruning_schedule': sparsity.PolynomialDecay(initial_sparsity=0.50,
                                                   final_sparsity=0.9,
                                                   begin_step=0,
                                                   end_step=end_step,
                                                   frequency=80)
    }


735


In [15]:
new_pruned_model = sparsity.prune_low_magnitude(loaded_model, **new_pruning_params)

opa=Adam(lr=learnrate, beta_1=0.9, beta_2=0.999, amsgrad=False)
new_pruned_model.compile(
    loss='categorical_crossentropy',#tf.keras.losses.categorical_crossentropy,
    optimizer=opa,
    metrics=['accuracy'])

Instructions for updating:
Please use `layer.add_weight` method instead.


In [16]:
callbacks = [
    sparsity.UpdatePruningStep()
    ]
history=new_pruned_model.fit_generator(train_generator,
                            steps_per_epoch=step_size_train,
                            epochs=newepochs,
                            verbose=1,
                            initial_epoch = 0,
                            callbacks=callbacks,
                            validation_data=validation_generator,
                            validation_steps=step_size_vali)

Epoch 1/15
49/49 [==============================] - 31s 630ms/step - loss: 0.1371 - accuracy: 0.9692 - val_loss: 0.3162 - val_accuracy: 0.8625
Epoch 2/15
49/49 [==============================] - 42s 864ms/step - loss: 0.1957 - accuracy: 0.9423 - val_loss: 0.8356 - val_accuracy: 0.6250
Epoch 3/15
49/49 [==============================] - 29s 593ms/step - loss: 0.1001 - accuracy: 0.9756 - val_loss: 0.2318 - val_accuracy: 0.9187
Epoch 4/15
49/49 [==============================] - 29s 596ms/step - loss: 0.2512 - accuracy: 0.9154 - val_loss: 0.2286 - val_accuracy: 0.9500
Epoch 5/15
49/49 [==============================] - 29s 599ms/step - loss: 0.2140 - accuracy: 0.9346 - val_loss: 2.5590 - val_accuracy: 0.4688
Epoch 6/15
49/49 [==============================] - 32s 648ms/step - loss: 0.2445 - accuracy: 0.9269 - val_loss: 0.2716 - val_accuracy: 0.9062
Epoch 7/15
49/49 [==============================] - 29s 592ms/step - loss: 0.2615 - accuracy: 0.9192 - val_loss: 0.6498 - val_accuracy: 0.7063

In [17]:
loss, acc=new_pruned_model.evaluate_generator(generator=validation_generator,
                                    steps=step_size_vali,
                                    verbose=0)
print("Accuracy: " ,acc)
print("Loss: ", loss)

Instructions for updating:
Please use Model.evaluate, which supports generators.
Accuracy:  0.9375
Loss:  0.1765105426311493


In [18]:
final_model = sparsity.strip_pruning(new_pruned_model)

In [19]:
prunedfile='ownmodel_pruW.h5'

In [20]:
print('Saving pruned model to: ', prunedfile)
models.save_model(final_model, prunedfile, 
                        include_optimizer=False)
model=models.load_model(prunedfile)
model.compile(optimizer=opa,loss='categorical_crossentropy',metrics=['accuracy'])
loss, acc=model.evaluate_generator(generator=validation_generator,
                                    steps=step_size_vali)
print("Accuracy: " ,acc)
print("Loss: ", loss)

Saving pruned model to:  ownmodel_pruW.h5
Accuracy:  0.9375
Loss:  0.1744621843099594


In [21]:
import numpy as np    
for i, w in enumerate(model.get_weights()):
    print('{} -- Total:{}, zero:{}'.format(
            model.weights[i].name,w.size,np.sum(w==0)/w.size*100)
         )

conv1/kernel:0 -- Total:1728, zero:89.98842592592592
conv1/bias:0 -- Total:64, zero:0.0
BN1/gamma:0 -- Total:64, zero:0.0
BN1/beta:0 -- Total:64, zero:0.0
BN1/moving_mean:0 -- Total:64, zero:0.0
BN1/moving_variance:0 -- Total:64, zero:0.0
conv2/kernel:0 -- Total:73728, zero:89.99972873263889
conv2/bias:0 -- Total:128, zero:0.0
BN2/gamma:0 -- Total:128, zero:0.0
BN2/beta:0 -- Total:128, zero:0.0
BN2/moving_mean:0 -- Total:128, zero:0.0
BN2/moving_variance:0 -- Total:128, zero:0.0
conv3/kernel:0 -- Total:147456, zero:89.99972873263889
conv3/bias:0 -- Total:128, zero:0.0
BN3/gamma:0 -- Total:128, zero:0.0
BN3/beta:0 -- Total:128, zero:0.0
BN3/moving_mean:0 -- Total:128, zero:0.0
BN3/moving_variance:0 -- Total:128, zero:0.0
conv4/kernel:0 -- Total:294912, zero:89.99972873263889
conv4/bias:0 -- Total:256, zero:0.0
BN4/gamma:0 -- Total:256, zero:0.0
BN4/beta:0 -- Total:256, zero:0.0
BN4/moving_mean:0 -- Total:256, zero:0.0
BN4/moving_variance:0 -- Total:256, zero:0.0
convf/kernel:0 -- Total:

In [22]:
import zipfile

zip3='ownsque_pruW.zip'
with zipfile.ZipFile(zip3, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(prunedfile)
print("Size of the pruned model before compression: %.2f KB" 
      % (os.path.getsize(prunedfile) / float(2**10)))
print("Size of the pruned model after compression: %.2f KB" 
      % (os.path.getsize(zip3) / float(2**10)))

Size of the pruned model before compression: 2086.86 KB
Size of the pruned model after compression: 404.86 KB


In [23]:
zip1='ownsque_ori.zip'
with zipfile.ZipFile(zip1, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(orifile)
print("Size of the unpruned model before compression: %.2f KB" % 
      (os.path.getsize(orifile) / float(2**10)))
print("Size of the unpruned model after compression: %.2f KB" % 
      (os.path.getsize(zip1) / float(2**10)))

Size of the unpruned model before compression: 2086.86 KB
Size of the unpruned model after compression: 1895.70 KB


In [24]:
test_generator.reset()
start_time = time.time()
pred=model.predict_generator(test_generator,
                             steps=step_size_test,
                             verbose=1)
print("--- %s seconds ---" % (time.time() - start_time))

173/173 [==============================] - 2s 11ms/step
--- 2.1210005283355713 seconds ---


In [25]:
predicted_class_indices=np.argmax(pred,axis=1)
#print(predicted_class_indices)

In [26]:
from sklearn.metrics import classification_report, confusion_matrix

In [27]:
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
print('precision = {0:.2f}'.format(precision_score(test_generator.classes, predicted_class_indices, average="macro")))
print('recall = {0:.2f}'.format(recall_score(test_generator.classes, predicted_class_indices, average="macro")))
print('f1 = {0:.2f}'.format(f1_score(test_generator.classes, predicted_class_indices, average="macro")))

precision = 0.95
recall = 0.96
f1 = 0.96


In [28]:
labels = train_generator.class_indices
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
ground_truth = test_generator.classes
fnames = test_generator.filenames
label2index = test_generator.class_indices

In [29]:
results=pd.DataFrame({"Filename":fnames,
                      "Predictions":predictions})
results.to_csv("resultsownnet.csv",index=False)

In [30]:
errors = np.where(predicted_class_indices != ground_truth)[0]
eresult=len(errors)/len(ground_truth)
print("No of errors = {}/{} = {}".format(len(errors),len(ground_truth),eresult))

No of errors = 9/173 = 0.05202312138728324


In [31]:
try:
  # %tensorflow_version only exists in Colab.
  import tensorflow.compat.v2 as tf
except Exception:
    pass
tf.enable_v2_behavior()
print("Eagerly enabled: ", tf.executing_eagerly())
print(tf.__version__)

Eagerly enabled:  True
2.3.0-dev20200515


In [32]:
m1='ownmodel_pruW.h5'
model=tf.keras.models.load_model(m1)
tflite_model_file = 'ownnet.tflite'
converter = tf.lite.TFLiteConverter.from_keras_model(model)
# converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
# converter.target_spec.supported_types = [tf.float16]
tflite_model = converter.convert()
with open(tflite_model_file, 'wb') as f:
    f.write(tflite_model)

Instructions for updating:
Simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: C:\Users\Bojie\AppData\Local\Temp\tmpzpetbq8v\assets


ConverterError: See console for info.
2020-05-16 17:48:30.914124: I tensorflow/lite/toco/import_tensorflow.cc:1324] Converting unsupported operation: FusedBatchNormV3
2020-05-16 17:48:30.914668: E tensorflow/core/framework/op_kernel.cc:1325] OpKernel ('op: "WrapDatasetVariant" device_type: "CPU"') for unknown op: WrapDatasetVariant
2020-05-16 17:48:30.914953: E tensorflow/core/framework/op_kernel.cc:1325] OpKernel ('op: "WrapDatasetVariant" device_type: "GPU" host_memory_arg: "input_handle" host_memory_arg: "output_handle"') for unknown op: WrapDatasetVariant
2020-05-16 17:48:30.915355: E tensorflow/core/framework/op_kernel.cc:1325] OpKernel ('op: "UnwrapDatasetVariant" device_type: "CPU"') for unknown op: UnwrapDatasetVariant
2020-05-16 17:48:30.915642: E tensorflow/core/framework/op_kernel.cc:1325] OpKernel ('op: "UnwrapDatasetVariant" device_type: "GPU" host_memory_arg: "input_handle" host_memory_arg: "output_handle"') for unknown op: UnwrapDatasetVariant
2020-05-16 17:48:30.916110:

In [ ]:
interpreter = tf.lite.Interpreter(model_path=str(tflite_model_file))
interpreter.allocate_tensors()
input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

In [ ]:
def evalmodel(test_generator, interpreter):
    pre1=[]
    for i in range(test_generator.n):
        x= next(test_generator)
        interpreter.set_tensor(input_index, x)
        interpreter.invoke()
        predictions = interpreter.get_tensor(output_index)
    #     print(predictions)
        predicted_class_indices1=np.argmax(predictions)#,axis=1
    #     print(predicted_class_indices1)
        pre1.append(predicted_class_indices1)
    return pre1

In [ ]:
predicted_class_indices=evalmodel(test_generator, interpreter)
print('Classification Report')
print(classification_report(test_generator.classes, predicted_class_indices, target_names=train_labels))